In [7]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder
from cosapp.drivers import NonLinearSolver, RunOnce, MonteCarlo, RunSingleCase
from cosapp.utils.distributions import Normal
import sys
sys.path.append('./model')
from Earth import Earth
import json
import numpy as np

earth = Earth("earth")

l = 10 #Rocket's length on the plot
dt = 0.05 #Time-step


from cosapp.utils.distributions import Normal
from cosapp.drivers import MonteCarlo
from cosapp.recorders import DataFrameRecorder

In [8]:

LOAD = True

if LOAD:
    with open("./include/init_rocket/rocket_dict.json", "r") as f:
        rocket_dict = json.load(f)
    l = rocket_dict['tube_length'] + rocket_dict['nose_length'] #Rocket's length on the plot
    angz = - rocket_dict['rocket_launch_angle']
    # Load the thrust.txt
    thrust = rocket_dict['motor']['samples']
    with open("model/Utility/thrust.txt", "w") as f:
        for i in range(len(thrust[0])):
            f.write(", ".join([str(point[i]) for point in thrust]))
            if i < len(thrust[0]) - 1:
                f.write("\n")

init = {
    'Traj.r' : np.array([-(l/2)*np.sin(angz), 0., (l/2)*np.cos(angz)]),
    'Rocket.Kin.v' : np.array([0,0,0]),
    'Rocket.Kin.ar' : 'initrot',
    'Rocket.Kin.av' : np.zeros(3),
    'Para.DynPar.r1' : np.array([0., 0., l/2]),
    'Para.DynPar.r2' : np.array([0., 0., l/2]),
    'Para.DynPar.v1' : np.array([0,0,0]),
    'Para.DynPar.v2' : np.array([0,0,0]),
    'Traj.ParaDepStatus': False
}
# rocket_dict parameters
if LOAD:
    init = {**init, 
            'Traj.r' : np.array([0,0, rocket_dict['rocket_cog'][0]]),
            'Para.DynPar.r1' : np.array([0., 0., rocket_dict['rocket_cog'][0]]),
            'Para.DynPar.r2' : np.array([0., 0., rocket_dict['rocket_cog'][0]]),
            'Rocket.CG': rocket_dict['rocket_cog'][0],
            'Rocket.l' : l,
            'Rocket.Mass.m' : rocket_dict['rocket_mass'],
            'Rocket.Mass.m0' : rocket_dict['rocket_mass'],
            'Rocket.Mass.Dm' : rocket_dict['rocket_prop_weight']/thrust[-1][0],
            'Rocket.Mass.lastEngineTime' : thrust[-1][0],
            'Rocket.Mass.I0_geom' : [rocket_dict['rocket_inertia'][i][i] for i in range(3)],

            'Rocket.Aero.Coefs.ln' : rocket_dict['nose_length'],
            'Rocket.Aero.Coefs.d' : 2*rocket_dict['tube_radius'],
            'Rocket.Aero.Coefs.NFins' : rocket_dict['fins_number'],
            'Rocket.Aero.Coefs.s' : rocket_dict['fins_s'],
            'Rocket.Aero.Coefs.Xt' : rocket_dict['fins_Xt'],
            'Rocket.Aero.Coefs.Cr' : rocket_dict['fins_Cr'],
            'Rocket.Aero.Coefs.Ct' : rocket_dict['fins_Ct'],
            'Rocket.Aero.Coefs.tf' : rocket_dict['fins_thickness'],
            'Rocket.Aero.Coefs.delta' : rocket_dict['delta'],

            'Wind.wind_on' : rocket_dict['wind_on'],
            # 'Wind.wind_average_speed' : rocket_dict['wind_average_speed'],

            'Para.l0' : rocket_dict['parachute_l0'],
            'Para.m1' : rocket_dict['parachute_weight'] + rocket_dict['ejected_nose_mass'],
            'Para.m2' : rocket_dict['rocket_mass'] - rocket_dict['ejected_nose_mass'],
            'Para.DynPar.S_ref' : rocket_dict['parachute_sref'],
            'Para.DynPar.Cd' : rocket_dict['parachute_Cd'],

            'Traj.parachute_deploy_method' : 0 if rocket_dict['parachute_deploy_method'] == 'velocity' else 1,
            'Traj.parachute_deploy_timer' : rocket_dict['parachute_deploy_timer'],
            }

In [14]:
def run_analysis(syst, draws=10, linear=True):
    syst.drivers.clear()  # Remove all drivers on the System

    runonce = syst.add_driver(RunOnce("runonce"))
    driver = syst.add_driver(RungeKutta(order=4, dt=dt))
    # driver.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r', 'initrot']))
    driver.add_child(NonLinearSolver('solver', factor=1.0))
    driver.time_interval = (0, 40)

    # Define a simulation scenario
    driver.set_scenario(
        init = init,
        stop = 'Para.DynPar.r2[2] < -1'
    )

    syst.run_drivers()

    # return driver.recorder.export_data()

    print("MONTECARLO")
    # Montecarlo
    syst.drivers.clear()
    montecarlo = syst.add_driver(MonteCarlo('mc'))
    montecarlo.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r', 'initrot']))
    montecarlo.add_child(driver)
    montecarlo.draws = draws
    # montecarlo.linear = linear

    # parameters for uncertainties in the data
    pitch_attr = syst.inwards.get_details('pitch_init')
    yaw_attr = syst.inwards.get_details('yaw_init')
    
    # Set the distribution around the current value
    pitch_attr.distribution = Normal(best=.2, worst=-0.02)
    yaw_attr.distribution = Normal(best=.3, worst=-0.3)

    montecarlo.add_random_variable('pitch_init')
    montecarlo.add_random_variable('yaw_init')

    # Computation
    syst.run_drivers()

    return montecarlo.recorder.export_data()

In [15]:

results = run_analysis(earth, draws=10, linear=False)

MONTECARLO


In [16]:
results

,Section,Status,Error code,Reference,Rocket.Kin.ar,Traj.r,initrot
0,,,0,0,"[1.8418955572286565e-06, -1.4802440955795837, ...","[2.781565202540398, -5.680039534635829e-07, 10...","[0.0, -1.4707963267948965, 0.09000000000000001]"
1,,,0,1,"[-2.061470080135557e-05, -1.4802440152841225, ...","[2.7703094685440033, 0.2500076182571723, 10.46...","[0.0, -1.6660302194019785, 0.16158576062259672]"
2,,,0,2,"[3.9271940995465404e-05, -1.6570940737995172, ...","[-2.6150839167812694, -0.42626489044302746, 10...","[0.0, -1.2755624341878145, 0.018414239377403288]"
3,,,0,3,"[-1.1854467610701468e-17, -1.3091729588261953,...","[8.069615764546153, 0.1486087720413232, 9.4815...","[0.0, -1.563027829714733, 0.05618178226272662]"
4,,,0,4,"[-0.00015755055927279104, -1.5640053734505448,...","[0.21213869759320514, 0.011935203484731434, 10...","[0.0, -1.1378228645241018, 0.21209026949929144]"
5,,,0,5,"[5.89534137757945e-17, -1.1959474666458996, 0....","[11.333389625021798, 2.4404080282581107, 8.273...","[0.0, -1.8037697890656912, 0.1238182177372734]"
6,,,0,6,"[1.0391589237529197e-06, -1.7791295707996448, ...","[-6.366840307131113, -0.7923821816348376, 9.89...","[0.0, -1.3785648238750599, -0.032090269499291404]"
7,,,0,7,"[5.658997167785273e-07, -1.3980237139278644, -...","[5.3123874075689175, -0.17054272021998085, 10....","[0.0, -1.6122747129804464, -0.004155709842326605]"
8,,,0,8,"[-6.705387561938318e-06, -1.6083466126584636, ...","[-1.153830154718134, 0.004795795418271609, 10....","[0.0, -1.214008027224915, 0.14187540826803496]"
9,,,0,9,"[-8.29751742730633e-17, -1.2577820296914208, 0...","[9.573176091647472, 1.3673888836535275, 8.9905...","[0.0, -1.914853965202851, 0.2528211340829166]"


In [17]:
traj = np.asarray(results['Traj.r'].tolist())
traj

array([[ 2.78156520e+00, -5.68003953e-07,  1.04610602e+01],
       [ 2.77030947e+00,  2.50007618e-01,  1.04610602e+01],
       [-2.61508392e+00, -4.26264890e-01,  1.04732020e+01],
       [ 8.06961576e+00,  1.48608772e-01,  9.48158308e+00],
       [ 2.12138698e-01,  1.19352035e-02,  1.05913409e+01],
       [ 1.13333896e+01,  2.44040803e+00,  8.27317247e+00],
       [-6.36684031e+00, -7.92382182e-01,  9.89268445e+00],
       [ 5.31238741e+00, -1.70542720e-01,  1.01128339e+01],
       [-1.15383015e+00,  4.79579542e-03,  1.05695831e+01],
       [ 9.57317609e+00,  1.36738888e+00,  8.99055836e+00]])

In [18]:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

traj = np.asarray(results['Traj.r'].tolist())

# Create the figure object
fig = make_subplots(rows=1, cols=1)
fig.layout.title = "Probability"
fig.layout.yaxis.title = 'Y Position'

fig.add_trace(
    go.Scatter(
        x=traj[:,0],
        y=traj[:,1],
        mode = 'markers'
    ),
    row = 1,
    col = 1,
)
fig.get_subplot(1, 1).xaxis.title = "X Position"

fig.show()